In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Fake News Detection – Exploratory Data Analysis (EDA)\n",
        "\n",
        "This notebook explores:\n",
        "- Fake/True dataset: title, text, subject, date\n",
        "- LIAR dataset: statement + metadata\n",
        "\n",
        "Adjust file paths as needed."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "from pathlib import Path\n",
        "import pandas as pd\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "\n",
        "ROOT = Path('..').resolve()\n",
        "DATA = ROOT / 'data' / 'raw'\n",
        "\n",
        "fake_path = DATA / 'Fake.csv'\n",
        "true_path = DATA / 'True.csv'\n",
        "liar_train = DATA / 'liar_train.csv'\n",
        "liar_valid = DATA / 'liar_valid.csv'\n",
        "liar_test  = DATA / 'liar_test.csv'\n",
        "\n",
        "fake_path, true_path, liar_train"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Load Fake/True Dataset"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "fake_df = pd.read_csv(fake_path)\n",
        "true_df = pd.read_csv(true_path)\n",
        "\n",
        "fake_df.columns = [c.strip().lower() for c in fake_df.columns]\n",
        "true_df.columns = [c.strip().lower() for c in true_df.columns]\n",
        "\n",
        "fake_df['label'] = 0\n",
        "true_df['label'] = 1\n",
        "\n",
        "df_ft = pd.concat([fake_df, true_df], ignore_index=True)\n",
        "df_ft[['title','text','subject','date','label']].head()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "df_ft['label'].value_counts()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Text Length Distributions"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "df_ft['title_len'] = df_ft['title'].fillna('').astype(str).str.len()\n",
        "df_ft['text_len']  = df_ft['text'].fillna('').astype(str).str.len()\n",
        "\n",
        "plt.figure()\n",
        "plt.hist(df_ft['title_len'], bins=60)\n",
        "plt.title('Title length (chars)')\n",
        "plt.xlabel('chars')\n",
        "plt.ylabel('count')\n",
        "plt.show()\n",
        "\n",
        "plt.figure()\n",
        "plt.hist(df_ft['text_len'], bins=60)\n",
        "plt.title('Body length (chars)')\n",
        "plt.xlabel('chars')\n",
        "plt.ylabel('count')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Top Subjects"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "df_ft['subject'] = df_ft['subject'].fillna('')\n",
        "df_ft['subject'].value_counts().head(20)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Load LIAR Dataset\n",
        "\n",
        "Expected columns by position:\n",
        "0 id, 1 label, 2 statement, 3 subject(s), 4 speaker, 5 job, 6 state, 7 party,\n",
        "8-12 history counts, 13 context."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def load_liar(path: Path, split: str) -> pd.DataFrame:\n",
        "    df = pd.read_csv(path)\n",
        "    if df.shape[1] != 14:\n",
        "        df = pd.read_csv(path, header=None)\n",
        "    df.columns = [\n",
        "        'id','liar_label','statement','subject','speaker','speaker_job','state','party',\n",
        "        'barely_true_counts','false_counts','half_true_counts','mostly_true_counts','pants_on_fire_counts','context'\n",
        "    ]\n",
        "    df['split'] = split\n",
        "    return df\n",
        "\n",
        "liar_tr = load_liar(liar_train, 'train')\n",
        "liar_va = load_liar(liar_valid, 'valid')\n",
        "liar_te = load_liar(liar_test, 'test')\n",
        "df_liar = pd.concat([liar_tr, liar_va, liar_te], ignore_index=True)\n",
        "df_liar.head()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "df_liar['liar_label'] = df_liar['liar_label'].astype(str).str.strip().str.lower()\n",
        "df_liar['liar_label'].value_counts()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Statement Length"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "df_liar['statement_len'] = df_liar['statement'].fillna('').astype(str).str.len()\n",
        "plt.figure()\n",
        "plt.hist(df_liar['statement_len'], bins=60)\n",
        "plt.title('LIAR statement length (chars)')\n",
        "plt.xlabel('chars')\n",
        "plt.ylabel('count')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Quick Checks for Potential Leakage\n",
        "\n",
        "Examples:\n",
        "- URLs embedded in the text\n",
        "- Publisher/source patterns\n",
        "- Extremely short texts"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "ft_has_url = df_ft['text'].fillna('').astype(str).str.contains('http', case=False).mean()\n",
        "liar_has_url = df_liar['statement'].fillna('').astype(str).str.contains('http', case=False).mean()\n",
        "ft_has_url, liar_has_url"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.10"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}
